In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from IPython.display import Image, display
import graphviz
import pydotplus

In [2]:
# https://www.datafiles.samhsa.gov/dataset/mental-health-client-level-data-2019-mh-cld-2019-ds0001
# https://www.datafiles.samhsa.gov/sites/default/files/MH-CLD-2019-DS0001-info-codebook.pdf - Describes the features of the dataset
# A dataset comprised of the mental health outcomes of people in mental health treatment facilities. 2019 statistics
# These facilities report to individual state administrative systems


behavioral_df = pd.read_csv('mhcld-puf-2019-csv.csv')
behavioral_df

,YEAR,AGE,EDUC,ETHNIC,RACE,GENDER,SPHSERVICE,CMPSERVICE,OPISERVICE,RTCSERVICE,...,ODDFLG,PDDFLG,PERSONFLG,SCHIZOFLG,ALCSUBFLG,OTHERDISFLG,STATEFIP,DIVISION,REGION,CASEID
0,2019,-9,-9,4,2,1,2,1,2,2,...,0,0,0,0,0,1,1,6,3,20190000001
1,2019,14,4,4,6,2,1,2,2,2,...,0,0,0,0,0,0,1,6,3,20190000002
2,2019,12,-9,4,3,2,1,1,2,2,...,0,0,0,1,0,0,1,6,3,20190000003
3,2019,10,-9,4,5,2,1,1,1,2,...,0,0,0,1,0,0,1,6,3,20190000004
4,2019,2,2,4,5,2,1,1,2,2,...,0,0,0,0,0,0,1,6,3,20190000005
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6362039,2019,5,-9,4,5,1,2,1,2,2,...,0,0,0,0,0,0,99,0,0,20196362040
6362040,2019,4,4,4,6,1,2,1,2,2,...,0,0,0,0,0,0,99,0,0,20196362041
6362041,2019,8,1,4,2,1,2,1,2,2,...,0,0,0,1,0,0,99,0,0,20196362042
6362042,2019,11,4,4,4,1,2,1,2,2,...,0,0,0,0,0,0,99,0,0,20196362043


In [3]:
behavioral_df = behavioral_df.drop(columns='YEAR')

# Remove values that are negative (i.e -9 corresponds to a missing value)
##behavioral_df[(behavioral_df > 0).all(1)] 

# Preserve order of the values for each feature besides Gender, Veteran since these are binary
# This table is made up of purely demographic data 


demographic_df = behavioral_df[['AGE', 'EDUC', 'ETHNIC', 'RACE', 'GENDER', 'MARSTAT', 'EMPLOY', 'VETERAN', 'DETNLF', 'LIVARAG', 'CASEID']]
demographic_df[(demographic_df > 0).all(1)]
demographic_df

,AGE,EDUC,ETHNIC,RACE,GENDER,MARSTAT,EMPLOY,VETERAN,DETNLF,LIVARAG,CASEID
0,-9,-9,4,2,1,-9,-9,-9,-9,-9,20190000001
1,14,4,4,6,2,4,-9,2,-9,3,20190000002
2,12,-9,4,3,2,4,-9,2,-9,-9,20190000003
3,10,-9,4,5,2,1,4,-9,-9,-9,20190000004
4,2,2,4,5,2,1,-9,-9,-9,2,20190000005
...,...,...,...,...,...,...,...,...,...,...,...
6362039,5,-9,4,5,1,-9,-9,2,-9,-9,20196362040
6362040,4,4,4,6,1,1,2,2,-9,2,20196362041
6362041,8,1,4,2,1,1,4,2,-9,2,20196362042
6362042,11,4,4,4,1,2,5,2,1,2,20196362043


In [9]:
# These variables account for locations where a client received mental health services

service_df = behavioral_df[['SPHSERVICE', 'CMPSERVICE', 'OPISERVICE', 'RTCSERVICE', 'IJSSERVICE']]
service_df[(service_df > 0).all(1)]
service_df[['SPHSERVICE', 'CMPSERVICE', 'OPISERVICE', 'RTCSERVICE', 'IJSSERVICE']] = service_df[['SPHSERVICE', 'CMPSERVICE', 'OPISERVICE', 'RTCSERVICE', 'IJSSERVICE']].replace(2, 0)
service_df[['SPHSERVICE', 'CMPSERVICE', 'OPISERVICE', 'RTCSERVICE', 'IJSSERVICE']] = service_df[['SPHSERVICE', 'CMPSERVICE', 'OPISERVICE', 'RTCSERVICE', 'IJSSERVICE']].replace(1, 1)

# Subset the service dataframe
service_subset_df = service_df[:100000]
service_subset_df = service_subset_df.append(service_df[100000:200000].where(behavioral_df['DEPRESSFLG'] == 1))
service_subset_df.head()

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


,SPHSERVICE,CMPSERVICE,OPISERVICE,RTCSERVICE,IJSSERVICE
0,0.0,1.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0
2,1.0,1.0,0.0,0.0,0.0
3,1.0,1.0,1.0,0.0,0.0
4,1.0,1.0,0.0,0.0,0.0


In [30]:
# IMPORTANT DATAFRAME: TABLE OF FEATURES OF TYPE OF MENTAL HEALTH DIAGNOSIS DURING THE 2019 REPORTING PERIOD OF THIS DATA
# values correpond to labels of the type of disorder a person was diagnosed with 


mh_diagnosis_kind_df = behavioral_df[['MH1', 'MH2', 'MH3', 'SUB', 'SMISED', 'SAP', 'CASEID']]
mh_diagnosis_kind_df['SAP'] = mh_diagnosis_kind_df['SAP'].replace(2, 0)
mh_diagnosis_kind_df[(mh_diagnosis_kind_df > 0).all(1)]
mh_diagnosis_kind_df

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,MH1,MH2,MH3,SUB,SMISED,SAP,CASEID
0,13,-9,-9,-9,-9,0,20190000001
1,7,-9,-9,-9,1,0,20190000002
2,11,-9,-9,-9,-9,0,20190000003
3,11,-9,-9,-9,1,0,20190000004
4,1,-9,-9,-9,2,0,20190000005
...,...,...,...,...,...,...,...
6362039,6,-9,-9,-9,1,-9,20196362040
6362040,2,-9,-9,-9,1,-9,20196362041
6362041,11,-9,-9,-9,1,-9,20196362042
6362042,7,-9,-9,-9,1,-9,20196362043


In [45]:
# DATAFRAME OF REPORTED DISORDERS. The disorders covered here are mentioned as the types of disorder diagnosis 
# made in the table above

reported_disorder_df = behavioral_df[['ANXIETYFLG', 'ADHDFLG', 'BIPOLARFLG', 'SCHIZOFLG', 'TRAUSTREFLG', 'CONDUCTFLG', 
                                    'DELIRDEMFLG', 'ODDFLG', 'PDDFLG', 'PERSONFLG', 'ALCSUBFLG', 'DEPRESSFLG', 'CASEID']]

# The variable below characterizes whether or not a patient has depression 
# DEPRESSION_DIAGNOSIS SERVES AS THE TARGET VARIABLE TO PREDICT
# NOTE: SAMPLES OF NO REPORTS OF DEPRESSION ARE 2X GREATER THAN THOSE REPORTS OF PATIENTS WITH DEPRESSION
# DATA IMBALANCE ISSUE 


reported_disorder_df

,ANXIETYFLG,ADHDFLG,BIPOLARFLG,SCHIZOFLG,TRAUSTREFLG,CONDUCTFLG,DELIRDEMFLG,ODDFLG,PDDFLG,PERSONFLG,ALCSUBFLG,DEPRESSFLG,CASEID
0,0,0,0,0,0,0,0,0,0,0,0,0,20190000001
1,0,0,0,0,0,0,0,0,0,0,0,1,20190000002
2,0,0,0,1,0,0,0,0,0,0,0,0,20190000003
3,0,0,0,1,0,0,0,0,0,0,0,0,20190000004
4,0,0,0,0,1,0,0,0,0,0,0,0,20190000005
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6362039,0,0,1,0,0,0,0,0,0,0,0,0,20196362040
6362040,1,0,0,0,0,0,0,0,0,0,0,0,20196362041
6362041,0,0,0,1,0,0,0,0,0,0,0,0,20196362042
6362042,0,0,0,0,0,0,0,0,0,0,0,1,20196362043


# Demographic Data Cleaning 
- The original dataset is broken up into 4 subsets where each subset contains a group of similar variables
- lets gain insight into the demographic makeup of the Client-Level Mental Health 2019 Report

- Retrieve class 1 points to avoid data imbalance issues

In [12]:
# Subset 100,000 rows from the dataset
demsubset_df = demographic_df[:100000]
demsubset_df = demsubset_df.append(demographic_df[100000:200000].where(behavioral_df['DEPRESSFLG'] == 1))
demsubset_df.index = np.arange(0, len(demsubset_df))
print('Number of datapoints is {}'.format(len(demsubset_df.values)))

demsubset_df = demsubset_df.replace(-9, np.nan)
demsubset_df['VETERAN'] = demsubset_df['VETERAN'].replace(2, 0)
demsubset_df['GENDER'] = demsubset_df['GENDER'].replace(2, 0)
demsubset_df.head() 

Number of datapoints is 200000


,AGE,EDUC,ETHNIC,RACE,GENDER,MARSTAT,EMPLOY,VETERAN,DETNLF,LIVARAG,CASEID
0,NaN,NaN,4.0,2.0,1.0,NaN,NaN,NaN,NaN,NaN,2.019000e+10
1,14.0,4.0,4.0,6.0,0.0,4.0,NaN,0.0,NaN,3.0,2.019000e+10
2,12.0,NaN,4.0,3.0,0.0,4.0,NaN,0.0,NaN,NaN,2.019000e+10
3,10.0,NaN,4.0,5.0,0.0,1.0,4.0,NaN,NaN,NaN,2.019000e+10
4,2.0,2.0,4.0,5.0,0.0,1.0,NaN,NaN,NaN,2.0,2.019000e+10


In [13]:
# Take a peak at the value counts for each value of each column
for col in demsubset_df.columns:
    print(f'{col} value counts', '\n')
    display(demsubset_df[col].value_counts(dropna=False))
    print('-------------------------------------------')

AGE value counts 



NaN     74078
1.0     16691
3.0     11303
2.0     10166
7.0      9314
8.0      9273
6.0      9161
12.0     8588
11.0     8549
9.0      8377
10.0     8312
14.0     7327
13.0     6480
4.0      6246
5.0      6135
Name: AGE, dtype: int64

-------------------------------------------
EDUC value counts 



NaN    96639
4.0    36703
2.0    34562
3.0    20434
5.0    10721
1.0      941
Name: EDUC, dtype: int64

-------------------------------------------
ETHNIC value counts 



...

-------------------------------------------
LIVARAG value counts 



NaN    110545
2.0     82217
3.0      5689
1.0      1549
Name: LIVARAG, dtype: int64

-------------------------------------------
CASEID value counts 



NaN             74064
2.019000e+10        1
2.019008e+10        1
2.019008e+10        1
2.019008e+10        1
                ...  
2.019004e+10        1
2.019004e+10        1
2.019004e+10        1
2.019004e+10        1
2.019020e+10        1
Name: CASEID, Length: 125937, dtype: int64

-------------------------------------------


In [14]:
# Look at demsubset_df na counts
demsubset_df.isna().sum()

AGE         74078
EDUC        96639
ETHNIC      76831
RACE        78453
GENDER      74103
MARSTAT     95826
EMPLOY     130667
VETERAN    109463
DETNLF     160572
LIVARAG    110545
CASEID      74064
dtype: int64

In [15]:
# Drop missing values for AGE, EDUC, ETHNIC, RACE, GENDER, EMPLOY, DETNLF, LIVARAG, VETERAN, and MARSTAT columns

demsubset_df.dropna(subset=['AGE', 'EDUC', 'ETHNIC', 'RACE', 'GENDER', 'MARSTAT', 'DETNLF', 'EMPLOY', 'LIVARAG', 
                           'VETERAN'], inplace=True)

In [16]:
demsubset_df.head()

,AGE,EDUC,ETHNIC,RACE,GENDER,MARSTAT,EMPLOY,VETERAN,DETNLF,LIVARAG,CASEID
28,3.0,3.0,4.0,5.0,0.0,1.0,5.0,0.0,2.0,2.0,2.019000e+10
32,10.0,3.0,4.0,5.0,1.0,4.0,5.0,0.0,1.0,2.0,2.019000e+10
34,11.0,2.0,4.0,5.0,1.0,4.0,5.0,0.0,1.0,2.0,2.019000e+10
36,11.0,3.0,4.0,5.0,0.0,2.0,5.0,0.0,1.0,2.0,2.019000e+10
38,9.0,4.0,4.0,3.0,1.0,4.0,5.0,0.0,1.0,2.0,2.019000e+10


In [17]:
# Confirm the correct na values were dropped from each specified column

for col in demsubset_df.columns:
    print(f'{col} value counts', '\n')
    display(demsubset_df[col].value_counts(dropna=False))
    print('-------------------------------------------')

AGE value counts 



14.0    2908
3.0     2602
12.0    2563
11.0    2558
10.0    2429
8.0     2396
9.0     2291
4.0     2283
7.0     2247
13.0    2164
6.0     2108
5.0     1410
Name: AGE, dtype: int64

-------------------------------------------
EDUC value counts 



4.0    12728
3.0     7556
2.0     3935
5.0     3423
1.0      317
Name: EDUC, dtype: int64

-------------------------------------------
ETHNIC value counts 



...

-------------------------------------------
LIVARAG value counts 



2.0    25042
3.0     2354
1.0      563
Name: LIVARAG, dtype: int64

-------------------------------------------
CASEID value counts 



2.019000e+10    1
2.019007e+10    1
2.019007e+10    1
2.019007e+10    1
2.019007e+10    1
               ..
2.019004e+10    1
2.019004e+10    1
2.019004e+10    1
2.019004e+10    1
2.019011e+10    1
Name: CASEID, Length: 27959, dtype: int64

-------------------------------------------


In [18]:
# Filling in names of values for multiple columns. Specify new dataframe for cleaned demographic data
demsubset_df['MARSTAT'].replace({1.0:'Never married',
                                 2.0: 'Now married',
                                 3.0: 'Separated',
                                 4.0: 'Divorced'}, inplace=True)

demsubset_df['EMPLOY'].replace({1.0:'Full Time',
                                 2.0: 'Part Time',
                                 3.0: 'Full Time/Part Time',
                                 4.0: 'Unemployed',
                                 5.0: 'Not in labor force'}, inplace=True)

demsubset_df['DETNLF'].replace({1.0:'Retired, disabled',
                                 2.0: 'Student',
                                 3.0: 'Homemaker',
                                 4.0: 'Sheltered/non-competitive employment',
                                 5.0: 'Other'}, inplace=True)

demsubset_df['RACE'].replace({1.0:'American Indian',
                                 2.0: 'Asian',
                                 3.0: 'Black or African American',
                                 4.0: 'Native Hawaiian or Other Pacific Islander',
                                 5.0: 'White',
                                 6.0: 'Two or more races'}, inplace=True)

demsubset_df['ETHNIC'].replace({1.0:'Mexican',
                                 2.0: 'Puerto Rican',
                                 3.0: 'Other Hispanic',
                                 4.0: 'Not Hispanic'}, inplace=True)

demsubset_df['LIVARAG'].replace({1.0:'Homeless',
                                 2.0: 'Private Residence',
                                 3.0: 'Other'}, inplace=True)

demsubset_df['VETERAN'].replace({1.0: 1,
                                 2.0: 0}, inplace=True)


demo_clean = demsubset_df.copy()

In [19]:
# Check that the values were renamed for each column
demo_clean.head()

,AGE,EDUC,ETHNIC,RACE,GENDER,MARSTAT,EMPLOY,VETERAN,DETNLF,LIVARAG,CASEID
28,3.0,3.0,Not Hispanic,White,0.0,Never married,Not in labor force,0.0,Student,Private Residence,2.019000e+10
32,10.0,3.0,Not Hispanic,White,1.0,Divorced,Not in labor force,0.0,"Retired, disabled",Private Residence,2.019000e+10
34,11.0,2.0,Not Hispanic,White,1.0,Divorced,Not in labor force,0.0,"Retired, disabled",Private Residence,2.019000e+10
36,11.0,3.0,Not Hispanic,White,0.0,Now married,Not in labor force,0.0,"Retired, disabled",Private Residence,2.019000e+10
38,9.0,4.0,Not Hispanic,Black or African American,1.0,Divorced,Not in labor force,0.0,"Retired, disabled",Private Residence,2.019000e+10


In [20]:
# Rename ambiguous columns

demo_clean.rename(columns={'EDUC': 'Education', 'MARSTAT': 'Marital Status', 'DETNLF': 'Not in Labor Force', 
                           'LIVARAG': 'Residential Status'},
                             inplace=True)

In [21]:
# Check for missing values of the cleaned dataframe
demo_clean.isna().sum()

AGE                   0
Education             0
ETHNIC                0
RACE                  0
GENDER                0
Marital Status        0
EMPLOY                0
VETERAN               0
Not in Labor Force    0
Residential Status    0
CASEID                0
dtype: int64

In [22]:
demo_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27959 entries, 28 to 114253
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   AGE                 27959 non-null  float64
 1   Education           27959 non-null  float64
 2   ETHNIC              27959 non-null  object 
 3   RACE                27959 non-null  object 
 4   GENDER              27959 non-null  float64
 5   Marital Status      27959 non-null  object 
 6   EMPLOY              27959 non-null  object 
 7   VETERAN             27959 non-null  float64
 8   Not in Labor Force  27959 non-null  object 
 9   Residential Status  27959 non-null  object 
 10  CASEID              27959 non-null  float64
dtypes: float64(5), object(6)
memory usage: 2.6+ MB


In [23]:
# Converting AGE, EEducation, and VETERAN columns to int type

convert_cols = ['AGE', 'Education', 'VETERAN']

for col in convert_cols:
    demo_clean[col] = demo_clean[col].astype('int64')

In [24]:
demo_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27959 entries, 28 to 114253
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   AGE                 27959 non-null  int64  
 1   Education           27959 non-null  int64  
 2   ETHNIC              27959 non-null  object 
 3   RACE                27959 non-null  object 
 4   GENDER              27959 non-null  float64
 5   Marital Status      27959 non-null  object 
 6   EMPLOY              27959 non-null  object 
 7   VETERAN             27959 non-null  int64  
 8   Not in Labor Force  27959 non-null  object 
 9   Residential Status  27959 non-null  object 
 10  CASEID              27959 non-null  float64
dtypes: float64(2), int64(3), object(6)
memory usage: 2.6+ MB


In [25]:
# Remove the CASEID column
demo_clean.drop(columns=['CASEID'], inplace=True)

In [26]:
demo_clean.columns

Index(['AGE', 'Education', 'ETHNIC', 'RACE', 'GENDER', 'Marital Status',
       'EMPLOY', 'VETERAN', 'Not in Labor Force', 'Residential Status'],
      dtype='object')

In [27]:
demo_clean.shape

(27959, 10)

In [28]:
# Re-index dataset
demo_clean.index = np.arange(0, len(demo_clean))

demo_clean.head()

,AGE,Education,ETHNIC,RACE,GENDER,Marital Status,EMPLOY,VETERAN,Not in Labor Force,Residential Status
0,3,3,Not Hispanic,White,0.0,Never married,Not in labor force,0,Student,Private Residence
1,10,3,Not Hispanic,White,1.0,Divorced,Not in labor force,0,"Retired, disabled",Private Residence
2,11,2,Not Hispanic,White,1.0,Divorced,Not in labor force,0,"Retired, disabled",Private Residence
3,11,3,Not Hispanic,White,0.0,Now married,Not in labor force,0,"Retired, disabled",Private Residence
4,9,4,Not Hispanic,Black or African American,1.0,Divorced,Not in labor force,0,"Retired, disabled",Private Residence


# Analyzing/Cleaning Mental Health Diagnosis Reports

In [32]:
# Replacing -9 values with nan
mh_diagnosis_kind_df = mh_diagnosis_kind_df.replace(-9, np.nan)

# Subset the mental health diagnosis dataframe
mh_subset_df = mh_diagnosis_kind_df[:100000]
mh_subset_df = mh_subset_df.append(mh_diagnosis_kind_df[100000:200000].where(behavioral_df['DEPRESSFLG'] == 1))
mh_subset_df.head()

,MH1,MH2,MH3,SUB,SMISED,SAP,CASEID
0,13.0,NaN,NaN,NaN,NaN,0.0,2.019000e+10
1,7.0,NaN,NaN,NaN,1.0,0.0,2.019000e+10
2,11.0,NaN,NaN,NaN,NaN,0.0,2.019000e+10
3,11.0,NaN,NaN,NaN,1.0,0.0,2.019000e+10
4,1.0,NaN,NaN,NaN,2.0,0.0,2.019000e+10


In [33]:
# Look at missing value counts for all columns
mh_subset_df.isna().sum()

MH1        74267
MH2       173811
MH3       194651
SUB       187962
SMISED     81374
SAP        74115
CASEID     74064
dtype: int64

In [34]:
# The columns MH2 and MH3 for the second and third respective diagnosis reports are high in missing values. 
# The column SUB for substance abuse reports is also unnecessary to keep

mh_subset_df = mh_subset_df.drop(columns=['MH2', 'MH3', 'SUB'])

In [35]:
# Observe value counts for each column/analyze missing value distributions

for col in mh_subset_df.columns:
    print(f'{col} value counts', '\n')
    display(mh_subset_df[col].value_counts(dropna=False))
    print('-------------------------------------------')

MH1 value counts 



NaN     74267
7.0     63760
11.0    14922
3.0     10513
6.0     10412
13.0     9297
1.0      7061
2.0      5791
8.0      1573
4.0       960
9.0       564
10.0      348
5.0       288
12.0      244
Name: MH1, dtype: int64

-------------------------------------------
SMISED value counts 



NaN    81374
1.0    66258
2.0    34885
3.0    17483
Name: SMISED, dtype: int64

-------------------------------------------
SAP value counts 



0.0    110790
NaN     74115
1.0     15095
Name: SAP, dtype: int64

-------------------------------------------
CASEID value counts 



NaN             74064
2.019000e+10        1
2.019008e+10        1
2.019008e+10        1
2.019008e+10        1
                ...  
2.019004e+10        1
2.019004e+10        1
2.019004e+10        1
2.019004e+10        1
2.019020e+10        1
Name: CASEID, Length: 125937, dtype: int64

-------------------------------------------


Quick observations: 
- The value 7.0 of the MH1 column represents the number of depressive disorder diagnoses for the first instance of diagnosis for a patient. Note, this value has the highest count out of all mental illness types
- The value 2.0 of the MH2 column represents the number of anxiety disorder diagnoses for the second instance of a diagnosis for a given patient. This value appears the most out of all other mental illness second diagnosis types (anxiety count > depression count) -> comorbidity

In [36]:
# Drop missing values across remaining columns

mh_subset_df.dropna(subset=['MH1', 'SMISED', 'SAP'], inplace=True)

In [37]:
mh_subset_df.head()

,MH1,SMISED,SAP,CASEID
1,7.0,1.0,0.0,2.019000e+10
3,11.0,1.0,0.0,2.019000e+10
4,1.0,2.0,0.0,2.019000e+10
5,1.0,1.0,1.0,2.019000e+10
6,13.0,2.0,0.0,2.019000e+10


- The MH1 column contains values accounting for the first diagnosis of various disorders when a client seeks mental health treatment for their first time

In [38]:
# Rename the values of the MH1 column

mh_subset_df['MH1'].replace({1:'Trauma Disorder', 2:'Anxiety disorders', 3:'ADHD', 
                            4:'Conduct disorders', 5:'Delirium, Dementia', 6:'Bipolar disorders',
                            7:'Depressive Disorders', 8:'Oppositional Defiant Disorders', 
                            9:'Pervasive Developmental Disorders', 10:'Personality disorders', 
                             11:'Schizophrenia or other psychosis', 12:'Alcohol or substance abuse disorders', 
                             13:'Other disorders'}, inplace=True)

In [39]:
# Rename the values of the SMISED column
mh_subset_df['SMISED'].replace({1:'Serious Mental Illness', 2:'Serious Emotional Disorder', 3:'Not serious'}, inplace=True)

In [40]:
# Rename value of SAP (Substance Abuse Problem) column
mh_subset_df['SAP'].replace({2: 0}, inplace=True)

In [41]:
# Look at value counts

for col in mh_subset_df.columns:
    print(f'{col} value counts', '\n')
    display(mh_subset_df[col].value_counts(dropna=False))
    print('-------------------------------------------')

MH1 value counts 



Depressive Disorders                    60559
Schizophrenia or other psychosis        14016
ADHD                                    10112
Bipolar disorders                        9433
Other disorders                          8758
Trauma Disorder                          6529
Anxiety disorders                        5207
Oppositional Defiant Disorders           1517
Conduct disorders                         927
Pervasive Developmental Disorders         526
Personality disorders                     334
Delirium, Dementia                        261
Alcohol or substance abuse disorders      240
Name: MH1, dtype: int64

-------------------------------------------
SMISED value counts 



Serious Mental Illness        66150
Serious Emotional Disorder    34856
Not serious                   17413
Name: SMISED, dtype: int64

-------------------------------------------
SAP value counts 



0.0    104641
1.0     13778
Name: SAP, dtype: int64

-------------------------------------------
CASEID value counts 



2.019000e+10    1
2.019009e+10    1
2.019009e+10    1
2.019009e+10    1
2.019009e+10    1
               ..
2.019004e+10    1
2.019004e+10    1
2.019004e+10    1
2.019004e+10    1
2.019020e+10    1
Name: CASEID, Length: 118419, dtype: int64

-------------------------------------------


- MH1 contains values accounting for depressive disorder cases of the patient population. Since these values directly address client depression, we remove these and instead add a column from the reported_subset_df which reports depression to serve as our target variable
- To avoid data imbalance issues, we select class 1 points in such a way that the target variable reaches a healthy equilibrium of data for each class 

In [46]:
reported_subset_df = reported_disorder_df[:100000]
reported_subset_df = reported_subset_df.append(reported_disorder_df[100000:200000].where(reported_disorder_df['DEPRESSFLG'] == 1))
reported_subset_df.head()

,ANXIETYFLG,ADHDFLG,BIPOLARFLG,SCHIZOFLG,TRAUSTREFLG,CONDUCTFLG,DELIRDEMFLG,ODDFLG,PDDFLG,PERSONFLG,ALCSUBFLG,DEPRESSFLG,CASEID
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.019000e+10
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.019000e+10
2,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.019000e+10
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.019000e+10
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.019000e+10


In [50]:
reported_subset_df.rename(columns={'DEPRESSFLG':'Diagnosed Depression'}, inplace=True)

In [51]:
reported_subset_df['Diagnosed Depression'].value_counts()

1.0    67692
0.0    58244
Name: Diagnosed Depression, dtype: int64

In [52]:
reported_subset_df.isna().sum()

ANXIETYFLG              74064
ADHDFLG                 74064
BIPOLARFLG              74064
SCHIZOFLG               74064
TRAUSTREFLG             74064
CONDUCTFLG              74064
DELIRDEMFLG             74064
ODDFLG                  74064
PDDFLG                  74064
PERSONFLG               74064
ALCSUBFLG               74064
Diagnosed Depression    74064
CASEID                  74064
dtype: int64

In [53]:
# Look at value counts

for col in reported_subset_df.columns:
    print(f'{col} value counts', '\n')
    display(reported_subset_df[col].value_counts(dropna=False))
    print('-------------------------------------------')

ANXIETYFLG value counts 



0.0    110020
NaN     74064
1.0     15916
Name: ANXIETYFLG, dtype: int64

-------------------------------------------
ADHDFLG value counts 



0.0    112940
NaN     74064
1.0     12996
Name: ADHDFLG, dtype: int64

-------------------------------------------
BIPOLARFLG value counts 



...

-------------------------------------------
Diagnosed Depression value counts 



NaN    74064
1.0    67692
0.0    58244
Name: Diagnosed Depression, dtype: int64

-------------------------------------------
CASEID value counts 



NaN             74064
2.019000e+10        1
2.019008e+10        1
2.019008e+10        1
2.019008e+10        1
                ...  
2.019004e+10        1
2.019004e+10        1
2.019004e+10        1
2.019004e+10        1
2.019020e+10        1
Name: CASEID, Length: 125937, dtype: int64

-------------------------------------------


In [54]:
# Increased number of rows to append to increase chance of obtaining more class 1 points after data cleaning and preprocessing

reported_subset_df.shape

(200000, 13)

In [56]:
mh_subset_df.shape

(118419, 4)

In [58]:
reported_subset_df = reported_subset_df.dropna()
reported_subset_df.shape

(125936, 13)

In [59]:
# Join the mh_subset_df and reported_subset_df to add the Target column to the mental illness diagnosis dataset
mh_diagnosis_df = pd.merge(mh_subset_df, reported_subset_df, how='left', on='CASEID')
mh_diagnosis_df.head()

,MH1,SMISED,SAP,CASEID,ANXIETYFLG,ADHDFLG,BIPOLARFLG,SCHIZOFLG,TRAUSTREFLG,CONDUCTFLG,DELIRDEMFLG,ODDFLG,PDDFLG,PERSONFLG,ALCSUBFLG,Diagnosed Depression
0,Depressive Disorders,Serious Mental Illness,0.0,2.019000e+10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,Schizophrenia or other psychosis,Serious Mental Illness,0.0,2.019000e+10,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Trauma Disorder,Serious Emotional Disorder,0.0,2.019000e+10,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Trauma Disorder,Serious Mental Illness,1.0,2.019000e+10,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Other disorders,Serious Emotional Disorder,0.0,2.019000e+10,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [60]:
mh_diagnosis_df.shape

(118419, 16)

In [61]:
mh_diagnosis_df['Diagnosed Depression'].value_counts()

1.0    64301
0.0    54118
Name: Diagnosed Depression, dtype: int64

In [62]:
mh_diagnosis_df.columns

Index(['MH1', 'SMISED', 'SAP', 'CASEID', 'ANXIETYFLG', 'ADHDFLG', 'BIPOLARFLG',
       'SCHIZOFLG', 'TRAUSTREFLG', 'CONDUCTFLG', 'DELIRDEMFLG', 'ODDFLG',
       'PDDFLG', 'PERSONFLG', 'ALCSUBFLG', 'Diagnosed Depression'],
      dtype='object')

In [63]:
# Keep original mh_subset_df columns with additional Diagnosed Depression column to serve as target variable

mh_diagnosis_df.drop(columns=['ANXIETYFLG', 'ADHDFLG', 'BIPOLARFLG', 'SCHIZOFLG', 'TRAUSTREFLG', 'CONDUCTFLG', 
                             'DELIRDEMFLG', 'ODDFLG', 'PDDFLG', 'PERSONFLG', 'ALCSUBFLG'], inplace=True)
mh_diagnosis_df.columns

Index(['MH1', 'SMISED', 'SAP', 'CASEID', 'Diagnosed Depression'], dtype='object')

In [64]:
# Checking values of dataframe

for col in mh_diagnosis_df.columns:
    print(f'{col} value counts', '\n')
    display(mh_diagnosis_df[col].value_counts(dropna=False))
    print('-------------------------------------------')

MH1 value counts 



Depressive Disorders                    60559
Schizophrenia or other psychosis        14016
ADHD                                    10112
Bipolar disorders                        9433
Other disorders                          8758
Trauma Disorder                          6529
Anxiety disorders                        5207
Oppositional Defiant Disorders           1517
Conduct disorders                         927
Pervasive Developmental Disorders         526
Personality disorders                     334
Delirium, Dementia                        261
Alcohol or substance abuse disorders      240
Name: MH1, dtype: int64

-------------------------------------------
SMISED value counts 



Serious Mental Illness        66150
Serious Emotional Disorder    34856
Not serious                   17413
Name: SMISED, dtype: int64

-------------------------------------------
SAP value counts 



...

-------------------------------------------
CASEID value counts 



2.019000e+10    1
2.019009e+10    1
2.019009e+10    1
2.019009e+10    1
2.019009e+10    1
               ..
2.019004e+10    1
2.019004e+10    1
2.019004e+10    1
2.019004e+10    1
2.019020e+10    1
Name: CASEID, Length: 118419, dtype: int64

-------------------------------------------
Diagnosed Depression value counts 



1.0    64301
0.0    54118
Name: Diagnosed Depression, dtype: int64

-------------------------------------------


In [65]:
# Replace the 'Depressive Disorders' values in the MH1 column with 'Missing'

mh_diagnosis_df['MH1'].replace({'Depressive Disorders': 'Missing'}, inplace=True)

In [66]:
# Check if nan values were removed

mh_diagnosis_df.isna().sum()

MH1                     0
SMISED                  0
SAP                     0
CASEID                  0
Diagnosed Depression    0
dtype: int64

In [67]:
# Rename the dataset to indicate it has been cleaned

diagnosis_clean = mh_diagnosis_df.copy()

In [68]:
diagnosis_clean.isna().sum()

MH1                     0
SMISED                  0
SAP                     0
CASEID                  0
Diagnosed Depression    0
dtype: int64

In [69]:
diagnosis_clean.shape

(118419, 5)

In [71]:
# Convert values to int
diagnosis_clean['SAP'] = diagnosis_clean['SAP'].astype('int64')
diagnosis_clean['Diagnosed Depression'] = diagnosis_clean['Diagnosed Depression'].astype('int64')

In [72]:
# Check changes
diagnosis_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 118419 entries, 0 to 118418
Data columns (total 5 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   MH1                   118419 non-null  object 
 1   SMISED                118419 non-null  object 
 2   SAP                   118419 non-null  int64  
 3   CASEID                118419 non-null  float64
 4   Diagnosed Depression  118419 non-null  int64  
dtypes: float64(1), int64(2), object(2)
memory usage: 5.4+ MB


In [73]:
# Remove the CASEID column
diagnosis_clean.drop(columns=['CASEID'], inplace=True)

In [74]:
diagnosis_clean.columns

Index(['MH1', 'SMISED', 'SAP', 'Diagnosed Depression'], dtype='object')

# Clean data of reported accounts of mental illness diagnosis 
- Each column is binary valued to illustrate whether or not a given client has been diagnosed with a particular disorder
- The reasons and symptomalogy behind the diagnosis is not provided in the official codebook

In [75]:
reported_subset_df.head()

,ANXIETYFLG,ADHDFLG,BIPOLARFLG,SCHIZOFLG,TRAUSTREFLG,CONDUCTFLG,DELIRDEMFLG,ODDFLG,PDDFLG,PERSONFLG,ALCSUBFLG,Diagnosed Depression,CASEID
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.019000e+10
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.019000e+10
2,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.019000e+10
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.019000e+10
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.019000e+10


In [76]:
# Drop the target column 

reported_subset_df.drop(columns='Diagnosed Depression', inplace=True)

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [77]:
# Rename columns to add more clarity 

reported_subset_df.rename(columns={'ANXIETYFLG': 'Anxiety disorder', 'SCHIZOFLG': 'Schizophrenia',
                          'TRAUSTREFLG': 'Trauma Disorders', 'CONDUCTFLG': 'Conduct Disorders', 'DELIRDEMFLG': 'Delirium/Dementia',
                          'ODDFLG': 'Oppositional Defiant Disorder', 'PDDFLG': 'Pervasive Development Disorder', 'PERSONFLG': 'Personality Disorder', 
                          'ALCSUBFLG': 'Alcohol/Substance Use Disorder'}, inplace=True)

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [78]:
# Check for missing values

reported_subset_df.isna().sum()

Anxiety disorder                  0
ADHDFLG                           0
BIPOLARFLG                        0
Schizophrenia                     0
Trauma Disorders                  0
Conduct Disorders                 0
Delirium/Dementia                 0
Oppositional Defiant Disorder     0
Pervasive Development Disorder    0
Personality Disorder              0
Alcohol/Substance Use Disorder    0
CASEID                            0
dtype: int64

In [79]:
# Rename the dataframe to show it has been cleaned

disorders_clean = reported_subset_df.copy()

In [80]:
# Look at value counts

for col in disorders_clean.columns:
    print(f'{col} value counts', '\n')
    display(disorders_clean[col].value_counts(dropna=False))
    print('-------------------------------------------')

Anxiety disorder value counts 



0.0    110020
1.0     15916
Name: Anxiety disorder, dtype: int64

-------------------------------------------
ADHDFLG value counts 



0.0    112940
1.0     12996
Name: ADHDFLG, dtype: int64

-------------------------------------------
BIPOLARFLG value counts 



...

-------------------------------------------
Alcohol/Substance Use Disorder value counts 



0.0    125502
1.0       434
Name: Alcohol/Substance Use Disorder, dtype: int64

-------------------------------------------
CASEID value counts 



2.019000e+10    1
2.019008e+10    1
2.019008e+10    1
2.019008e+10    1
2.019008e+10    1
               ..
2.019004e+10    1
2.019004e+10    1
2.019004e+10    1
2.019004e+10    1
2.019020e+10    1
Name: CASEID, Length: 125936, dtype: int64

-------------------------------------------


In [81]:
# Remove the CASEID column
disorders_clean.drop(columns=['CASEID'], inplace=True)

In [82]:
disorders_clean.columns

Index(['Anxiety disorder', 'ADHDFLG', 'BIPOLARFLG', 'Schizophrenia',
       'Trauma Disorders', 'Conduct Disorders', 'Delirium/Dementia',
       'Oppositional Defiant Disorder', 'Pervasive Development Disorder',
       'Personality Disorder', 'Alcohol/Substance Use Disorder'],
      dtype='object')

- The majority of client responses indicate that a very small portion of the population (100,000 rows) was diagnosed with a mental 
illness in the year 2019

# Clean column in-patient/out-patient setting data
- These fields identify where a given client received mental health services from
- Examples of Treatment facilities include: State psychiatric services, Residential treatment centers, Jail services, Private provides, etc..

In [83]:
print(service_subset_df.shape)
service_subset_df.head()

(200000, 5)


,SPHSERVICE,CMPSERVICE,OPISERVICE,RTCSERVICE,IJSSERVICE
0,0.0,1.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0
2,1.0,1.0,0.0,0.0,0.0
3,1.0,1.0,1.0,0.0,0.0
4,1.0,1.0,0.0,0.0,0.0


In [84]:
# Renaming columns
service_subset_df.rename(columns={'SPHSERVICE': 'State psychiatric hospital', 
                         'CMPSERVICE': 'Community Mental Health Center', 'OPISERVICE': 'Other psychiatric inpatient',
                        'RTCSERVICE': 'Residential Treatment Center', 'IJSSERVICE': 'justice system services'},
                        inplace=True)

In [85]:
service_subset_df.isna().sum()

State psychiatric hospital        74064
Community Mental Health Center    74064
Other psychiatric inpatient       74064
Residential Treatment Center      74064
justice system services           74064
dtype: int64

In [88]:
# Remove na values
service_subset_df = service_subset_df.dropna()
service_subset_df.isna().sum()

State psychiatric hospital        0
Community Mental Health Center    0
Other psychiatric inpatient       0
Residential Treatment Center      0
justice system services           0
dtype: int64

In [89]:
# Rename dataframe to show it has been cleaned

services_clean = service_subset_df.copy()

In [90]:
# Look at value counts

for col in services_clean.columns:
    print(f'{col} value counts', '\n')
    display(services_clean[col].value_counts(dropna=False))
    print('-------------------------------------------')

State psychiatric hospital value counts 



0.0    125050
1.0       886
Name: State psychiatric hospital, dtype: int64

-------------------------------------------
Community Mental Health Center value counts 



1.0    125223
0.0       713
Name: Community Mental Health Center, dtype: int64

-------------------------------------------
Other psychiatric inpatient value counts 



...

-------------------------------------------
Residential Treatment Center value counts 



0.0    125245
1.0       691
Name: Residential Treatment Center, dtype: int64

-------------------------------------------
justice system services value counts 



0.0    125781
1.0       155
Name: justice system services, dtype: int64

-------------------------------------------


In [91]:
services_clean.shape

(125936, 5)

# Aggregate clean subset-dataframes to one table

In [92]:
clean_df = pd.concat([demo_clean, diagnosis_clean, disorders_clean, services_clean], axis=1)

clean_df.head()

,AGE,Education,ETHNIC,RACE,GENDER,Marital Status,EMPLOY,VETERAN,Not in Labor Force,Residential Status,...,Delirium/Dementia,Oppositional Defiant Disorder,Pervasive Development Disorder,Personality Disorder,Alcohol/Substance Use Disorder,State psychiatric hospital,Community Mental Health Center,Other psychiatric inpatient,Residential Treatment Center,justice system services
0,3.0,3.0,Not Hispanic,White,0.0,Never married,Not in labor force,0.0,Student,Private Residence,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,10.0,3.0,Not Hispanic,White,1.0,Divorced,Not in labor force,0.0,"Retired, disabled",Private Residence,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,11.0,2.0,Not Hispanic,White,1.0,Divorced,Not in labor force,0.0,"Retired, disabled",Private Residence,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
3,11.0,3.0,Not Hispanic,White,0.0,Now married,Not in labor force,0.0,"Retired, disabled",Private Residence,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0
4,9.0,4.0,Not Hispanic,Black or African American,1.0,Divorced,Not in labor force,0.0,"Retired, disabled",Private Residence,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0


In [94]:
clean_df.shape

(139253, 30)

In [95]:
clean_df['Diagnosed Depression'].value_counts()

1.0    64301
0.0    54118
Name: Diagnosed Depression, dtype: int64

In [96]:
clean_df.isna().sum()

AGE                               111294
Education                         111294
ETHNIC                            111294
RACE                              111294
GENDER                            111294
Marital Status                    111294
EMPLOY                            111294
VETERAN                           111294
Not in Labor Force                111294
Residential Status                111294
MH1                                20834
SMISED                             20834
SAP                                20834
Diagnosed Depression               20834
Anxiety disorder                   13317
ADHDFLG                            13317
BIPOLARFLG                         13317
Schizophrenia                      13317
Trauma Disorders                   13317
Conduct Disorders                  13317
Delirium/Dementia                  13317
Oppositional Defiant Disorder      13317
Pervasive Development Disorder     13317
Personality Disorder               13317
Alcohol/Substanc

In [97]:
# Renaming columns

clean_df.rename(columns={'ETHNIC': 'Ethnicity', 'EMPLOY': 'Employment Status'}, inplace=True)

In [98]:
# To avoid issues with inconsistent data types, we replace 0 (na) values in categorical columns with 'Missing'

fillna_cols = ['Ethnicity', 'RACE', 'Marital Status', 'Employment Status', 'Not in Labor Force', 'Residential Status']
for col in fillna_cols:
    clean_df[col] = clean_df[col].replace(np.nan, 'Missing')

In [99]:
clean_df['Ethnicity'].unique()

array(['Not Hispanic', 'Puerto Rican', 'Other Hispanic', 'Mexican',
       'Missing'], dtype=object)

In [100]:
for col in clean_df.columns:
    print(f'{col} value counts', '\n')
    display(clean_df[col].value_counts(dropna=False))
    print('-------------------------------------------')

AGE value counts 



NaN     111294
14.0      2908
3.0       2602
12.0      2563
11.0      2558
10.0      2429
8.0       2396
9.0       2291
4.0       2283
7.0       2247
13.0      2164
6.0       2108
5.0       1410
Name: AGE, dtype: int64

-------------------------------------------
Education value counts 



NaN    111294
4.0     12728
3.0      7556
2.0      3935
5.0      3423
1.0       317
Name: Education, dtype: int64

-------------------------------------------
Ethnicity value counts 



...

-------------------------------------------
Residential Treatment Center value counts 



0.0    125245
NaN     13317
1.0       691
Name: Residential Treatment Center, dtype: int64

-------------------------------------------
justice system services value counts 



0.0    125781
NaN     13317
1.0       155
Name: justice system services, dtype: int64

-------------------------------------------


In [101]:
clean_df1 = clean_df.dropna()
clean_df1.shape

(27959, 30)

In [103]:
clean_df1['Diagnosed Depression'].value_counts()

0.0    16172
1.0    11787
Name: Diagnosed Depression, dtype: int64

In [104]:
clean_df1.shape

(27959, 30)

In [106]:
# Total number of training samples
train_total = clean_df1.shape[0]
print('Total number of training samples: {}'.format(train_total))

Total number of training samples: 27959


In [127]:
# Save to csv file to load in separate notebook for modeling
clean_df1.to_csv('CLEAN_MHCLD.csv')